In [ ]:
%%capture
#Installs Unsloth, Xformers (Flash Attention) and all other packages!
!pip install "unsloth[cu121-torch240] @ git+https://github.com/unslothai/unsloth.git"
!pip install  "xformers==0.0.28" "trl<0.9.0" peft accelerate bitsandbytes
# To ensure CUDA support, install PyTorch with CUDA first (CUDA 11.7 version here):

In [ ]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Meta-Llama-3.1-8B",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
==((====))==  Unsloth 2024.10.3: Fast Llama patching. Transformers = 4.44.2.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.4.1+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.28. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Unsloth: We fixed a gradient accumulation bug, but it seems like you don't have the latest transformers version!
Please update transformers via:
`pip uninstall transformers -y && pip install --upgrade --no-cache-dir "git+https://github.com/huggingface/transformers.git"`


In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth 2024.10.3 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [ ]:
!pip install datasets
!pip uninstall transformers -y && pip install --upgrade --no-cache-dir "git+https://github.com/huggingface/transformers.git
# alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

# ### prompt:
# {}
# ### Response:
# {}"""
# EOS_TOKEN_ID = tokenizer.eos_token_id

# from datasets import load_dataset


# def format_prompt(example):
#   prompt = example['prompt']
#   response = example['response']
#   texts = []
#   for prompts , responses in zip(prompt, response):
#     text = alpaca_prompt.format(prompts, responses) + tokenizer.eos_token
#     texts.append(text)
#   return {"text": texts, }

# pass

# dataset = load_dataset("csv",data_files = "prompts_responses1.csv", split="train" )
# dataset = dataset.map(format_prompt)

from datasets import load_dataset

# Define Alpaca prompt format
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### prompt:
{}
### Response:
{}"""
EOS_TOKEN_ID = tokenizer.eos_token_id

# Function to format each example
def format_prompt(example):
    # Format the prompt and response using the Alpaca template
    prompt = example['prompt']  # Single prompt
    response = example['response']  # Single response
    text = alpaca_prompt.format(prompt, response) + tokenizer.eos_token
    return {"text": text}

# Load dataset
dataset = load_dataset("csv", data_files="prompts_responses1.csv", split="train")

# Map the formatting function over the dataset
dataset = dataset.map(format_prompt)




/bin/bash: -c: line 1: unexpected EOF while looking for matching `"'
/bin/bash: -c: line 2: syntax error: unexpected end of file


In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported
from datasets import load_dataset



trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=dataset,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    dataset_num_proc=2,
    packing=False,  # Can make training 5x faster for short sequences.
    args=TrainingArguments(
        per_device_train_batch_size=2,
        gradient_accumulation_steps=4,
        warmup_steps=5,
        max_steps=60,
        learning_rate=2e-4,
        fp16=True,
        bf16=False,
        logging_steps=1,
        optim="adamw_8bit",
        weight_decay=0.01,
        lr_scheduler_type="linear",
        seed=3407,
        output_dir="outputs",
    ),
)

trainer_stats  = trainer.train()

max_steps is given, it will override any value given in num_train_epochs


**** Unsloth: Please use our fixed gradient_accumulation_steps by updating transformers and Unsloth!


==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 89,712 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 60
 "-____-"     Number of trainable parameters = 41,943,040
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: tarek-omar492 (tarek-omar492-suez-canal-university). Use `wandb login --relogin` to force relogin


Step,Training Loss
1,3.219000
2,2.984400
3,3.206300
4,3.030700
5,2.680300
6,2.571800
7,2.245300
8,1.975500
9,1.869800
10,1.458600


In [ ]:
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    alpaca_prompt.format(
        "Where can I find the online listing for the building in Greater Cairo  /  Bait El Watan El Asasy?", # instruction
         # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 64, use_cache = True)
tokenizer.batch_decode(outputs)

['<|begin_of_text|>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### prompt:\nWhere can I find the online listing for the building in Greater Cairo  /  Bait El Watan El Asasy?\n### Response:\nThe online listing for the building in Greater Cairo  /  Bait El Watan El Asasy is available at: https://aqarmap.com.eg/en/listing/4794956-for-sale-cairo-new-cairo-compounds-bait-el-watan-el-asasy/<|end_of_text|>']

In [ ]:
model.save_pretrained("lora_model") # Local saving
tokenizer.save_pretrained("lora_model")

('lora_model/tokenizer_config.json',
 'lora_model/special_tokens_map.json',
 'lora_model/tokenizer.json')

In [ ]:
!huggingface-cli login



    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) y
Token is valid (permission: fineGrained).
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub.
Run the following command in your termin

In [ ]:
from huggingface_hub import upload_folder

# Upload your model folder to the Hugging Face Hub
upload_folder(
    folder_path="lora_model",  # Path to the model folder
    repo_id="tarek009/my_little_broker",  # Your repo name
    commit_message="Initial model upload"
)




  0%|          | 0/1 [00:00<?, ?it/s]

adapter_model.safetensors:   0%|          | 0.00/168M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/tarek009/my_little_broker/commit/f185fa193045ba5d4751939869343089769c2226', commit_message='Initial model upload', commit_description='', oid='f185fa193045ba5d4751939869343089769c2226', pr_url=None, pr_revision=None, pr_num=None)